In [1]:
#Imports


import pandas as pd
from pandas import json_normalize
import requests 
import json
import os
from dotenv import load_dotenv #conda install -c conda-forge python-dotenv
import time
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [2]:
# I want the companies from City 1: Barcelona
# I want the companies from City 2: Amsterdam
# I want the companies from City 3: San Francisco

# I will scrape them from https://gamedevmap.com/index.php?location={CITYNAME}

In [2]:
#SCRAPE 1: BARCELONA
city = "Barcelona"
url_game_barcelona = f"https://gamedevmap.com/index.php?location={city}"
html_bcn = requests.get(url_game_barcelona)
soup = BeautifulSoup(html_bcn.content, "html.parser")
tags = soup.find_all("b")
bcn_companies = [i.getText() for i in tags]
bcn_companies = bcn_companies[6:]

In [83]:
'''reg_company = r">(\w+|\d+)<"
reg_temp = r">([A-Za-z\\s])<"

bcn_companies = re.findall(reg_company, str(tags))
bcn_companies = bcn_companies[5:]

print(len(bcn_companies), bcn_companies)'''

'reg_company = r">(\\w+|\\d+)<"\nreg_temp = r">([A-Za-z\\s])<"\n\nbcn_companies = re.findall(reg_company, str(tags))\nbcn_companies = bcn_companies[5:]\n\nprint(len(bcn_companies), bcn_companies)'

In [3]:
#SCRAPE 2: AMSTERDAM
city = "Amsterdam"
url_game_amsterdam = f"https://gamedevmap.com/index.php?location={city}"
html_ams = requests.get(url_game_amsterdam)
soup = BeautifulSoup(html_ams.content, "html.parser")
tags = soup.find_all("b")
ams_companies = [i.getText() for i in tags]
ams_companies = ams_companies[6:]

In [4]:
#SCRAPE 3: SAN FRANCISCO
city = "San%20Francisco"
url_game_sanfrancisco = f"https://gamedevmap.com/index.php?location={city}"
html_sf = requests.get(url_game_sanfrancisco)
soup = BeautifulSoup(html_sf.content, "html.parser")
tags = soup.find_all("b")
sf_companies = [i.getText() for i in tags]
sf_companies = sf_companies[6:]


In [10]:
bcn_companies

['2 Awesome Studio',
 '5 Ants',
 'A Crowd of Monsters',
 'Abylight',
 'Abylight Studios',
 'Alike Studio',
 'Altered Matter',
 'Anarkade',
 'Appnormals Team',
 'Awex Studios',
 'Beautiful Glitch',
 'Beautifun Games',
 'Bloompix Studios',
 'Corsegames',
 'Dark Curry',
 'Digital Legends',
 'Divertap',
 'DreamSoftGames',
 'Drop of Pixel',
 'Endflame Studios',
 'Final Boss Games',
 'Frozen Shard',
 'GameHouse Spain',
 'Gameloft',
 'GSN Games',
 'Herobeat Studios',
 'Hexworks (CI Games)',
 'Infinigon Games',
 'Jandusoft',
 'King.com (Activision)',
 'Larian Studios',
 'Life Zero Games',
 'Like a Boss Games',
 'Lince Works',
 'Little Miracles Games',
 'Lusho Games',
 'Manduka Games',
 'Nomada Studio',
 'Novarama',
 'Omnidrone',
 'Outfit7',
 'Pantumaca Barcelona',
 'Petoons Studio',
 'PiX Juegos',
 'Pixel Cream',
 'Playstark',
 'Psious',
 'Scopely',
 'SmileGate',
 'SoccerDream',
 'Social Point',
 'Starloop Studios',
 'Studio Koba',
 'Tangelo Games',
 'Tanuki Game Studio',
 'TikTak Games',
 'Ti

In [8]:
import requests

url = "https://api.foursquare.com/v3/places/nearby?query=2%20Ton%20Studio"

headers = {
    "accept": "application/json",
    "Authorization": "fsq3+fZhukazIdsdQYC8kjNOyqoI3tfZlCBgktME1Z2NElo="
}

response = requests.get(url, headers=headers).json()



In [9]:
response

{'results': [{'fsq_id': '4d814df8a4e237042fecc6cc',
   'categories': [],
   'chains': [],
   'distance': 465,
   'geocodes': {'main': {'latitude': 41.39298, 'longitude': 2.159198}},
   'link': '/v3/places/4d814df8a4e237042fecc6cc',
   'location': {'address': 'Provença, 229',
    'admin_region': 'Cataluña',
    'country': 'ES',
    'cross_street': '',
    'formatted_address': 'Provença, 229, 08008 Barcelona Catalonia',
    'locality': 'Barcelona',
    'postcode': '08008',
    'region': 'Catalonia'},
   'name': 'Studio 229',
   'related_places': {},
   'timezone': 'Europe/Madrid'},
  {'fsq_id': '4ea7d2ba46902b1f956d416e',
   'categories': [{'id': 11130,
     'name': 'Office Building',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/default_',
      'suffix': '.png'}}],
   'chains': [],
   'distance': 1515,
   'geocodes': {'main': {'latitude': 41.402283, 'longitude': 2.156188}},
   'link': '/v3/places/4ea7d2ba46902b1f956d416e',
   'location': {'address': 'Virtut, 